# Módulo 1: Introducción a la Minería de Datos## Validación y Evaluación de Modelos[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/husseinlopez/diplomadoIA/blob/main/M1-6_Ejercicios_Validacion.ipynb)**Diplomado en Inteligencia Artificial**  **Dr. Irvin Hussein López Nava**  **CICESE - UABC**---

## Objetivos de esta sesión1. **Comprender la importancia de la partición de datos** para evaluar generalización2. **Implementar diferentes esquemas de validación**: Hold-out, k-Fold CV, Leave-One-Out3. **Aplicar métricas de regresión**: MAE, MSE, RMSE, R²4. **Aplicar métricas de clasificación**: Accuracy, Precision, Recall, F1, ROC-AUC, MCC5. **Evitar data leakage** en el pipeline de evaluación6. **Interpretar resultados** considerando sesgo y varianza

## Estructura del notebook### Parte 1: Esquemas de Partición y Validación* Hold-out simple* k-Fold Cross-Validation* Leave-One-Out Cross-Validation* Estratificación### Parte 2: Evaluación de Modelos de Regresión* Métricas: MAE, MSE, RMSE, R²* Comparación de modelos* Análisis de residuos### Parte 3: Evaluación de Modelos de Clasificación* Matriz de confusión* Métricas: Accuracy, Precision, Recall, F1* Curva ROC y AUC* Matthews Correlation Coefficient (MCC)### Parte 4: Comparación de Modelos y Mejores Prácticas* Variabilidad del desempeño* Data leakage (cómo evitarlo)* Selección de métricas según contexto

---## 0. Configuración del EntornoImportaremos las bibliotecas necesarias para validación y evaluación.

In [ ]:
# Manejo de datosimport numpy as npimport pandas as pdfrom scipy import stats# Visualizaciónimport matplotlib.pyplot as pltimport seaborn as snsimport plotly.graph_objects as gofrom plotly.subplots import make_subplots# Configuración de visualizaciónplt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")%matplotlib inline# Reproducibilidadnp.random.seed(42)# Ignorar warningsimport warningswarnings.filterwarnings('ignore')print("✓ Bibliotecas básicas importadas")

In [ ]:
# Machine Learningfrom sklearn.model_selection import (    train_test_split,     cross_val_score,     cross_validate,    KFold,     StratifiedKFold,     LeaveOneOut)# Modelosfrom sklearn.linear_model import LinearRegression, Ridge, LogisticRegressionfrom sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifierfrom sklearn.ensemble import RandomForestRegressor, RandomForestClassifierfrom sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifierfrom sklearn.svm import SVR, SVC# Preprocesamientofrom sklearn.preprocessing import StandardScalerfrom sklearn.pipeline import Pipeline# Métricas de regresiónfrom sklearn.metrics import (    mean_absolute_error,    mean_squared_error,    r2_score)# Métricas de clasificaciónfrom sklearn.metrics import (    accuracy_score,    precision_score,    recall_score,    f1_score,    confusion_matrix,    classification_report,    roc_curve,    roc_auc_score,    matthews_corrcoef)# Datasetsfrom sklearn.datasets import (    load_diabetes,    load_breast_cancer,    make_classification,    make_regression)print("✓ Bibliotecas de ML importadas correctamente")

---# Parte 1: Esquemas de Partición y ValidaciónLa partición de datos es fundamental para estimar la capacidad de generalización del modelo.

## 1.1 ¿Por qué particionar los datos?**Problema fundamental**:- El modelo se ajusta minimizando el error en los datos de entrenamiento- Si evaluamos en los mismos datos, el error será **optimista**- No sabremos si el modelo **generaliza** a datos nuevos**Solución**:- Separar datos en **entrenamiento** y **prueba**- Entrenar solo con training- Evaluar solo con test (datos "no vistos")

## 1.2 Hold-Out SimpleLa estrategia más básica: una sola partición.

In [ ]:
# Crear dataset sintético para regresiónnp.random.seed(42)X_reg, y_reg = make_regression(    n_samples=200,     n_features=10,     n_informative=8,    noise=10,     random_state=42)print("="*80)print("HOLD-OUT SIMPLE")print("="*80)# Partición 70-30X_train, X_test, y_train, y_test = train_test_split(    X_reg, y_reg,     test_size=0.3,     random_state=42)print(f"\nTotal de datos: {len(X_reg)}")print(f"  Training: {len(X_train)} ({100*len(X_train)/len(X_reg):.0f}%)")print(f"  Test: {len(X_test)} ({100*len(X_test)/len(X_reg):.0f}%)")# Entrenar modelo simplemodel = LinearRegression()model.fit(X_train, y_train)# Predeciry_train_pred = model.predict(X_train)y_test_pred = model.predict(X_test)# Calcular errorestrain_mse = mean_squared_error(y_train, y_train_pred)test_mse = mean_squared_error(y_test, y_test_pred)train_r2 = r2_score(y_train, y_train_pred)test_r2 = r2_score(y_test, y_test_pred)print(f"\nResultados:")print(f"  MSE Training: {train_mse:.2f}")print(f"  MSE Test: {test_mse:.2f}")print(f"  R² Training: {train_r2:.3f}")print(f"  R² Test: {test_r2:.3f}")print(f"\n⚠️  Limitación: El resultado depende de la partición específica")print(f"   Cambiar random_state da resultados diferentes")

### Variabilidad del Hold-OutVeamos cómo cambia el desempeño con diferentes particiones.

In [ ]:
# Ejecutar hold-out con diferentes semillasresults = []for seed in range(30):    X_train, X_test, y_train, y_test = train_test_split(        X_reg, y_reg, test_size=0.3, random_state=seed    )        model = LinearRegression()    model.fit(X_train, y_train)    y_pred = model.predict(X_test)        mse = mean_squared_error(y_test, y_pred)    r2 = r2_score(y_test, y_pred)        results.append({'seed': seed, 'MSE': mse, 'R²': r2})df_results = pd.DataFrame(results)# Visualizaciónfig, axes = plt.subplots(1, 2, figsize=(14, 5))# MSEax = axes[0]ax.hist(df_results['MSE'], bins=15, alpha=0.7, color='steelblue', edgecolor='black')ax.axvline(df_results['MSE'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df_results["MSE"].mean():.2f}')ax.set_xlabel('MSE en Test')ax.set_ylabel('Frecuencia')ax.set_title('Variabilidad del MSE\n(30 particiones diferentes)', fontweight='bold')ax.legend()ax.grid(alpha=0.3)# R²ax = axes[1]ax.hist(df_results['R²'], bins=15, alpha=0.7, color='green', edgecolor='black')ax.axvline(df_results['R²'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df_results["R²"].mean():.3f}')ax.set_xlabel('R² en Test')ax.set_ylabel('Frecuencia')ax.set_title('Variabilidad del R²\n(30 particiones diferentes)', fontweight='bold')ax.legend()ax.grid(alpha=0.3)plt.tight_layout()plt.show()print(f"MSE: μ = {df_results['MSE'].mean():.2f}, σ = {df_results['MSE'].std():.2f}")print(f"R²:  μ = {df_results['R²'].mean():.3f}, σ = {df_results['R²'].std():.3f}")print(f"\n💡 Varianza alta → Hold-out simple no es confiable")

## 1.3 k-Fold Cross-ValidationReduce la varianza dividiendo los datos en k subconjuntos (folds).

In [ ]:
# k-Fold CV con k=5print("="*80)print("K-FOLD CROSS-VALIDATION (k=5)")print("="*80)kfold = KFold(n_splits=5, shuffle=True, random_state=42)model = LinearRegression()# cross_validate devuelve múltiples métricascv_results = cross_validate(    model, X_reg, y_reg,    cv=kfold,    scoring=['neg_mean_squared_error', 'r2'],    return_train_score=True)# Convertir a positivo (sklearn usa negative MSE)train_mse = -cv_results['train_neg_mean_squared_error']test_mse = -cv_results['test_neg_mean_squared_error']train_r2 = cv_results['train_r2']test_r2 = cv_results['test_r2']print(f"\nResultados por fold:")print(f"{'Fold':<10} {'Train MSE':<15} {'Test MSE':<15} {'Train R²':<15} {'Test R²':<15}")print("-" * 70)for i in range(5):    print(f"{i+1:<10} {train_mse[i]:<15.2f} {test_mse[i]:<15.2f} {train_r2[i]:<15.3f} {test_r2[i]:<15.3f}")print("\nEstadísticas agregadas:")print(f"  Test MSE: {test_mse.mean():.2f} ± {test_mse.std():.2f}")print(f"  Test R²:  {test_r2.mean():.3f} ± {test_r2.std():.3f}")print(f"\n✓ Cada observación se usa para entrenamiento y prueba")print(f"✓ Estimación más estable que hold-out simple")# Visualizaciónfig, axes = plt.subplots(1, 2, figsize=(14, 5))# MSE por foldax = axes[0]x = np.arange(1, 6)ax.bar(x - 0.2, train_mse, width=0.4, label='Training', alpha=0.7, color='steelblue')ax.bar(x + 0.2, test_mse, width=0.4, label='Test', alpha=0.7, color='orange')ax.axhline(test_mse.mean(), color='red', linestyle='--', linewidth=2, label=f'Test μ={test_mse.mean():.2f}')ax.set_xlabel('Fold')ax.set_ylabel('MSE')ax.set_title('MSE por Fold (5-Fold CV)', fontweight='bold')ax.set_xticks(x)ax.legend()ax.grid(alpha=0.3, axis='y')# R² por foldax = axes[1]ax.bar(x - 0.2, train_r2, width=0.4, label='Training', alpha=0.7, color='steelblue')ax.bar(x + 0.2, test_r2, width=0.4, label='Test', alpha=0.7, color='orange')ax.axhline(test_r2.mean(), color='red', linestyle='--', linewidth=2, label=f'Test μ={test_r2.mean():.3f}')ax.set_xlabel('Fold')ax.set_ylabel('R²')ax.set_title('R² por Fold (5-Fold CV)', fontweight='bold')ax.set_xticks(x)ax.legend()ax.grid(alpha=0.3, axis='y')plt.tight_layout()plt.show()

## 1.4 Leave-One-Out Cross-Validation (LOOCV)Caso extremo donde k = n (cada observación es un fold).

In [ ]:
# LOOCV (solo con subset pequeño por costo computacional)print("="*80)print("LEAVE-ONE-OUT CROSS-VALIDATION")print("="*80)# Usar solo primeras 100 observaciones para estabilidadX_small = X_reg[:100]y_small = y_reg[:100]loo = LeaveOneOut()# Usar modelo más robusto para LOOCVmodel = Ridge(alpha=1.0)# LOOCV con manejo de errorestry:    scores = cross_val_score(model, X_small, y_small, cv=loo, scoring='r2')        # Filtrar NaN si los hay    scores_valid = scores[~np.isnan(scores)]        if len(scores_valid) == 0:        print(f"\n⚠️  Todos los scores son NaN. Usando MAE en su lugar...")        scores = cross_val_score(model, X_small, y_small, cv=loo, scoring='neg_mean_absolute_error')        scores = -scores  # Convertir a positivo        metric_name = 'MAE'    else:        scores = scores_valid        metric_name = 'R²'        print(f"\nDatos: {len(X_small)} observaciones")    print(f"Iteraciones: {loo.get_n_splits(X_small)} (una por observación)")    print(f"\n{metric_name} por observación:")    print(f"  Media: {scores.mean():.3f}")    print(f"  Std: {scores.std():.3f}")    print(f"  Min: {scores.min():.3f}")    print(f"  Max: {scores.max():.3f}")        print(f"\n⚠️  LOOCV:")    print(f"   ✓ Bajo sesgo (entrena con n-1 datos)")    print(f"   ✗ Alta varianza")    print(f"   ✗ Alto costo computacional (n iteraciones)")    print(f"   → Usar solo con datasets pequeños")        # Visualización    fig, ax = plt.subplots(1, 1, figsize=(10, 5))    ax.hist(scores, bins=20, alpha=0.7, color='purple', edgecolor='black')    ax.axvline(scores.mean(), color='red', linestyle='--', linewidth=2,               label=f'Media: {scores.mean():.3f}')    ax.set_xlabel(f'{metric_name} Score')    ax.set_ylabel('Frecuencia')    ax.set_title(f'Distribución de {metric_name} en LOOCV\n({len(scores)} iteraciones)',                 fontweight='bold')    ax.legend()    ax.grid(alpha=0.3)    plt.tight_layout()    plt.show()    except Exception as e:    print(f"\n⚠️  Error en LOOCV: {str(e)}")    print(f"\nLOOCV puede ser inestable con pocos datos y regresión lineal.")    print(f"En la práctica, k-Fold CV (k=5 o k=10) es más robusto.")

## 1.5 Estratificación en ClasificaciónPreservar proporciones de clase en cada fold.

In [ ]:
# Crear dataset desbalanceado para clasificaciónX_class, y_class = make_classification(    n_samples=200,    n_features=10,    n_informative=8,    n_classes=2,    weights=[0.7, 0.3],  # 70% clase 0, 30% clase 1    random_state=42)print("="*80)print("ESTRATIFICACIÓN EN CLASIFICACIÓN")print("="*80)print(f"\nDistribución original:")unique, counts = np.unique(y_class, return_counts=True)for cls, count in zip(unique, counts):    print(f"  Clase {cls}: {count} ({100*count/len(y_class):.1f}%)")# CV SIN estratificaciónprint(f"\n--- K-Fold CV SIN estratificación ---")kfold_no_strat = KFold(n_splits=5, shuffle=True, random_state=42)for i, (train_idx, test_idx) in enumerate(kfold_no_strat.split(X_class), 1):    y_test_fold = y_class[test_idx]    class_1_pct = 100 * (y_test_fold == 1).sum() / len(y_test_fold)    print(f"  Fold {i}: {class_1_pct:.1f}% clase 1 en test")# CV CON estratificaciónprint(f"\n--- K-Fold CV CON estratificación ---")kfold_strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)for i, (train_idx, test_idx) in enumerate(kfold_strat.split(X_class, y_class), 1):    y_test_fold = y_class[test_idx]    class_1_pct = 100 * (y_test_fold == 1).sum() / len(y_test_fold)    print(f"  Fold {i}: {class_1_pct:.1f}% clase 1 en test")print(f"\n✓ Estratificación preserva las proporciones originales")print(f"✓ Esencial con clases desbalanceadas")

---# Parte 2: Evaluación de Modelos de RegresiónMétricas para problemas donde la variable objetivo es continua.

## 2.1 Dataset: DiabetesUsaremos el dataset clásico de diabetes para predecir progresión de la enfermedad.

In [ ]:
# Cargar datasetdiabetes = load_diabetes()X_diabetes = diabetes.datay_diabetes = diabetes.targetprint("="*80)print("DATASET: Diabetes")print("="*80)print(f"\nObservaciones: {X_diabetes.shape[0]}")print(f"Features: {X_diabetes.shape[1]}")print(f"\nVariable objetivo: Progresión de diabetes (continua)")print(f"  Min: {y_diabetes.min():.1f}")print(f"  Max: {y_diabetes.max():.1f}")print(f"  Media: {y_diabetes.mean():.1f}")print(f"  Std: {y_diabetes.std():.1f}")# ParticiónX_train_diab, X_test_diab, y_train_diab, y_test_diab = train_test_split(    X_diabetes, y_diabetes, test_size=0.3, random_state=42)print(f"\nPartición 70-30:")print(f"  Training: {len(X_train_diab)}")print(f"  Test: {len(X_test_diab)}")

## 2.2 Métricas de RegresiónCompararemos MAE, MSE, RMSE y R² en diferentes modelos.

In [ ]:
def evaluate_regression_model(model, X_train, X_test, y_train, y_test, model_name):    """    Evalúa un modelo de regresión con múltiples métricas    """    # Entrenar    model.fit(X_train, y_train)        # Predecir    y_train_pred = model.predict(X_train)    y_test_pred = model.predict(X_test)        # Calcular métricas    metrics = {        'Model': model_name,        'Train MAE': mean_absolute_error(y_train, y_train_pred),        'Test MAE': mean_absolute_error(y_test, y_test_pred),        'Train MSE': mean_squared_error(y_train, y_train_pred),        'Test MSE': mean_squared_error(y_test, y_test_pred),        'Train RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),        'Test RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),        'Train R²': r2_score(y_train, y_train_pred),        'Test R²': r2_score(y_test, y_test_pred)    }        return metrics, y_test_pred# Modelos a compararmodels = {    'Linear Regression': LinearRegression(),    'Ridge (α=1.0)': Ridge(alpha=1.0),    'Decision Tree': DecisionTreeRegressor(max_depth=5, random_state=42),    'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42),    'KNN (k=5)': KNeighborsRegressor(n_neighbors=5)}results = []predictions = {}print("="*80)print("COMPARACIÓN DE MODELOS - REGRESIÓN")print("="*80)for name, model in models.items():    metrics, y_pred = evaluate_regression_model(        model, X_train_diab, X_test_diab, y_train_diab, y_test_diab, name    )    results.append(metrics)    predictions[name] = y_pred    print(f"✓ {name}")df_results_reg = pd.DataFrame(results)print(f"\n{df_results_reg.to_string(index=False)}")

### Visualización de ResultadosComparemos las métricas y las predicciones.

In [ ]:
# Visualización de métricasfig, axes = plt.subplots(2, 2, figsize=(16, 12))metrics_to_plot = [    ('Test MAE', 'MAE en Test', 'steelblue'),    ('Test RMSE', 'RMSE en Test', 'orange'),    ('Test R²', 'R² en Test', 'green'),]for idx, (metric, title, color) in enumerate(metrics_to_plot):    ax = axes[idx // 2, idx % 2]        values = df_results_reg[metric].values    models_names = df_results_reg['Model'].values        bars = ax.barh(models_names, values, color=color, alpha=0.7, edgecolor='black')    ax.set_xlabel(metric, fontsize=11)    ax.set_title(title, fontweight='bold', fontsize=13)    ax.grid(alpha=0.3, axis='x')        # Añadir valores    for bar, val in zip(bars, values):        width = bar.get_width()        ax.text(width, bar.get_y() + bar.get_height()/2,               f' {val:.2f}', ha='left', va='center', fontsize=10, fontweight='bold')# Predicciones vs Real (mejor modelo por R²)ax = axes[1, 1]best_model_name = df_results_reg.loc[df_results_reg['Test R²'].idxmax(), 'Model']y_pred_best = predictions[best_model_name]ax.scatter(y_test_diab, y_pred_best, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)ax.plot([y_test_diab.min(), y_test_diab.max()],         [y_test_diab.min(), y_test_diab.max()],         'r--', linewidth=2, label='Predicción perfecta')ax.set_xlabel('Valores Reales', fontsize=11)ax.set_ylabel('Valores Predichos', fontsize=11)ax.set_title(f'Predicciones vs Reales\n(Mejor modelo: {best_model_name})', fontweight='bold', fontsize=13)ax.legend()ax.grid(alpha=0.3)plt.suptitle('Comparación de Modelos de Regresión', fontsize=16, fontweight='bold')plt.tight_layout()plt.show()print(f"\n🏆 Mejor modelo por R²: {best_model_name}")print(f"   Test R²: {df_results_reg.loc[df_results_reg['Test R²'].idxmax(), 'Test R²']:.3f}")

## 2.3 Análisis de ResiduosLos residuos revelan patrones de error del modelo.

In [ ]:
# Análisis de residuos del mejor modelobest_model_idx = df_results_reg['Test R²'].idxmax()best_model_name = df_results_reg.loc[best_model_idx, 'Model']y_pred_best = predictions[best_model_name]residuals = y_test_diab - y_pred_bestfig, axes = plt.subplots(1, 3, figsize=(18, 5))# Distribución de residuosax = axes[0]ax.hist(residuals, bins=20, alpha=0.7, color='steelblue', edgecolor='black')ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Cero')ax.axvline(residuals.mean(), color='orange', linestyle='--', linewidth=2, label=f'Media: {residuals.mean():.2f}')ax.set_xlabel('Residuos')ax.set_ylabel('Frecuencia')ax.set_title('Distribución de Residuos', fontweight='bold')ax.legend()ax.grid(alpha=0.3)# Residuos vs Prediccionesax = axes[1]ax.scatter(y_pred_best, residuals, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)ax.axhline(0, color='red', linestyle='--', linewidth=2)ax.set_xlabel('Valores Predichos')ax.set_ylabel('Residuos')ax.set_title('Residuos vs Predicciones', fontweight='bold')ax.grid(alpha=0.3)# Q-Q plotax = axes[2]stats.probplot(residuals, dist="norm", plot=ax)ax.set_title('Q-Q Plot (normalidad de residuos)', fontweight='bold')ax.grid(alpha=0.3)plt.suptitle(f'Análisis de Residuos: {best_model_name}', fontsize=16, fontweight='bold')plt.tight_layout()plt.show()print(f"Estadísticas de residuos:")print(f"  Media: {residuals.mean():.2f} (debe estar cerca de 0)")print(f"  Std: {residuals.std():.2f}")print(f"  Min: {residuals.min():.2f}")print(f"  Max: {residuals.max():.2f}")

---# Parte 3: Evaluación de Modelos de ClasificaciónMétricas para problemas donde la variable objetivo es categórica.

## 3.1 Dataset: Breast CancerUsaremos el dataset Wisconsin Breast Cancer (clasificación binaria).

In [ ]:
# Cargar datasetcancer = load_breast_cancer()X_cancer = cancer.datay_cancer = cancer.targetprint("="*80)print("DATASET: Wisconsin Breast Cancer")print("="*80)print(f"\nObservaciones: {X_cancer.shape[0]}")print(f"Features: {X_cancer.shape[1]}")print(f"\nClases:")unique, counts = np.unique(y_cancer, return_counts=True)for cls, count, name in zip(unique, counts, cancer.target_names):    print(f"  {cls} ({name}): {count} ({100*count/len(y_cancer):.1f}%)")# Partición estratificadaX_train_canc, X_test_canc, y_train_canc, y_test_canc = train_test_split(    X_cancer, y_cancer, test_size=0.3, random_state=42, stratify=y_cancer)print(f"\nPartición estratificada 70-30:")print(f"  Training: {len(X_train_canc)}")print(f"  Test: {len(X_test_canc)}")# Verificar estratificaciónprint(f"\nDistribución en test:")unique, counts = np.unique(y_test_canc, return_counts=True)for cls, count in zip(unique, counts):    print(f"  Clase {cls}: {count} ({100*count/len(y_test_canc):.1f}%)")

## 3.2 Matriz de ConfusiónLa base para todas las métricas de clasificación.

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names, model_name):    """    Visualiza la matriz de confusión    """    cm = confusion_matrix(y_true, y_pred)        fig, ax = plt.subplots(1, 1, figsize=(8, 6))        # Heatmap    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',                 xticklabels=class_names, yticklabels=class_names,                cbar_kws={'label': 'Cuenta'}, ax=ax)        ax.set_ylabel('Clase Real', fontsize=12)    ax.set_xlabel('Clase Predicha', fontsize=12)    ax.set_title(f'Matriz de Confusión\n{model_name}', fontweight='bold', fontsize=14)        # Añadir anotaciones TN, FP, FN, TP    tn, fp, fn, tp = cm.ravel()    annotations = [        (0, 0, f'TN={tn}'),        (1, 0, f'FP={fp}'),        (0, 1, f'FN={fn}'),        (1, 1, f'TP={tp}')    ]        for x, y, text in annotations:        ax.text(x + 0.5, y + 0.75, text, ha='center', va='center',               fontsize=10, color='darkred', fontweight='bold')        plt.tight_layout()    return fig, cm# Entrenar modelo simplemodel_lr = LogisticRegression(max_iter=10000, random_state=42)model_lr.fit(X_train_canc, y_train_canc)y_pred_lr = model_lr.predict(X_test_canc)# Matriz de confusiónfig_cm, cm = plot_confusion_matrix(y_test_canc, y_pred_lr, cancer.target_names, 'Logistic Regression')plt.show()# Métricas básicastn, fp, fn, tp = cm.ravel()print("="*80)print("MÉTRICAS DESDE LA MATRIZ DE CONFUSIÓN")print("="*80)print(f"\nVerdaderos Negativos (TN): {tn}")print(f"Falsos Positivos (FP):     {fp}  ← Error Tipo I")print(f"Falsos Negativos (FN):     {fn}  ← Error Tipo II")print(f"Verdaderos Positivos (TP): {tp}")

## 3.3 Métricas de ClasificaciónCalculemos todas las métricas principales.

In [ ]:
def calculate_classification_metrics(y_true, y_pred, y_pred_proba=None):    """    Calcula todas las métricas de clasificación    """    metrics = {        'Accuracy': accuracy_score(y_true, y_pred),        'Precision': precision_score(y_true, y_pred),        'Recall': recall_score(y_true, y_pred),        'F1-Score': f1_score(y_true, y_pred),        'MCC': matthews_corrcoef(y_true, y_pred)    }        if y_pred_proba is not None:        metrics['ROC-AUC'] = roc_auc_score(y_true, y_pred_proba)        return metrics# Métricas para Logistic Regressiony_pred_proba_lr = model_lr.predict_proba(X_test_canc)[:, 1]metrics_lr = calculate_classification_metrics(y_test_canc, y_pred_lr, y_pred_proba_lr)print("="*80)print("MÉTRICAS DE CLASIFICACIÓN - Logistic Regression")print("="*80)for metric, value in metrics_lr.items():    print(f"{metric:15s}: {value:.4f}")print(f"\nInterpretación:")print(f"  • Accuracy:  {metrics_lr['Accuracy']:.1%} de predicciones correctas")print(f"  • Precision: {metrics_lr['Precision']:.1%} de positivos predichos son correctos")print(f"  • Recall:    {metrics_lr['Recall']:.1%} de positivos reales fueron detectados")print(f"  • F1-Score:  Media armónica de Precision y Recall")print(f"  • MCC:       Correlación entre predicción y realidad [-1, 1]")print(f"  • ROC-AUC:   Área bajo la curva ROC [0.5, 1.0]")

## 3.4 Curva ROC y AUCLa curva ROC evalúa el desempeño variando el umbral de clasificación.

In [ ]:
# Calcular curva ROCfpr, tpr, thresholds = roc_curve(y_test_canc, y_pred_proba_lr)roc_auc = roc_auc_score(y_test_canc, y_pred_proba_lr)# Visualizaciónfig, axes = plt.subplots(1, 2, figsize=(16, 6))# Curva ROCax = axes[0]ax.plot(fpr, tpr, linewidth=3, label=f'Logistic Regression (AUC = {roc_auc:.3f})', color='steelblue')ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Clasificador aleatorio (AUC = 0.5)', alpha=0.5)ax.set_xlabel('False Positive Rate (1 - Specificity)', fontsize=12)ax.set_ylabel('True Positive Rate (Sensitivity)', fontsize=12)ax.set_title('Curva ROC', fontweight='bold', fontsize=14)ax.legend(fontsize=11)ax.grid(alpha=0.3)ax.set_xlim([-0.05, 1.05])ax.set_ylim([-0.05, 1.05])# Umbral óptimo# Criterio: maximizar TPR - FPRoptimal_idx = np.argmax(tpr - fpr)optimal_threshold = thresholds[optimal_idx]ax.plot(fpr[optimal_idx], tpr[optimal_idx], 'ro', markersize=10,         label=f'Umbral óptimo: {optimal_threshold:.3f}')ax.legend(fontsize=11)# Distribución de probabilidadesax = axes[1]y_pred_proba_class0 = y_pred_proba_lr[y_test_canc == 0]y_pred_proba_class1 = y_pred_proba_lr[y_test_canc == 1]ax.hist(y_pred_proba_class0, bins=30, alpha=0.6, color='red', label='Clase 0 (Maligno)', edgecolor='black')ax.hist(y_pred_proba_class1, bins=30, alpha=0.6, color='green', label='Clase 1 (Benigno)', edgecolor='black')ax.axvline(0.5, color='black', linestyle='--', linewidth=2, label='Umbral por defecto: 0.5')ax.axvline(optimal_threshold, color='orange', linestyle='--', linewidth=2, label=f'Umbral óptimo: {optimal_threshold:.3f}')ax.set_xlabel('Probabilidad predicha (clase 1)', fontsize=12)ax.set_ylabel('Frecuencia', fontsize=12)ax.set_title('Distribución de Probabilidades Predichas', fontweight='bold', fontsize=14)ax.legend(fontsize=10)ax.grid(alpha=0.3)plt.suptitle('Análisis ROC y Probabilidades', fontsize=16, fontweight='bold')plt.tight_layout()plt.show()print(f"ROC-AUC: {roc_auc:.4f}")print(f"Interpretación: Probabilidad de que el modelo asigne mayor score a un")print(f"                positivo real que a un negativo real")print(f"\nUmbral óptimo: {optimal_threshold:.3f} (vs 0.5 por defecto)")

## 3.5 Comparación de Múltiples ModelosEvaluemos diferentes clasificadores.

In [ ]:
def evaluate_classifier(model, X_train, X_test, y_train, y_test, model_name):    """    Evalúa un modelo de clasificación con todas las métricas    """    # Entrenar    model.fit(X_train, y_train)        # Predecir    y_pred = model.predict(X_test)    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None        # Métricas    metrics = {        'Model': model_name,        'Accuracy': accuracy_score(y_test, y_pred),        'Precision': precision_score(y_test, y_pred),        'Recall': recall_score(y_test, y_pred),        'F1-Score': f1_score(y_test, y_pred),        'MCC': matthews_corrcoef(y_test, y_pred)    }        if y_pred_proba is not None:        metrics['ROC-AUC'] = roc_auc_score(y_test, y_pred_proba)    else:        metrics['ROC-AUC'] = np.nan        return metrics, y_pred, y_pred_proba# Modelos a compararclassifiers = {    'Logistic Regression': LogisticRegression(max_iter=10000, random_state=42),    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5),    'SVM (linear)': SVC(kernel='linear', probability=True, random_state=42)}results_class = []predictions_class = {}probas_class = {}print("="*80)print("COMPARACIÓN DE CLASIFICADORES")print("="*80)for name, model in classifiers.items():    metrics, y_pred, y_proba = evaluate_classifier(        model, X_train_canc, X_test_canc, y_train_canc, y_test_canc, name    )    results_class.append(metrics)    predictions_class[name] = y_pred    probas_class[name] = y_proba    print(f"✓ {name}")df_results_class = pd.DataFrame(results_class)print(f"\n{df_results_class.to_string(index=False)}")# Identificar mejor modelobest_f1_idx = df_results_class['F1-Score'].idxmax()best_model_name = df_results_class.loc[best_f1_idx, 'Model']print(f"\n🏆 Mejor modelo por F1-Score: {best_model_name}")print(f"   F1-Score: {df_results_class.loc[best_f1_idx, 'F1-Score']:.4f}")

### Visualización Comparativa

In [ ]:
# Visualización de métricasfig, axes = plt.subplots(2, 3, figsize=(18, 10))axes = axes.ravel()metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'MCC', 'ROC-AUC']colors = ['steelblue', 'orange', 'green', 'red', 'purple', 'brown']for idx, (metric, color) in enumerate(zip(metrics_names, colors)):    ax = axes[idx]        values = df_results_class[metric].values    models = df_results_class['Model'].values        bars = ax.barh(models, values, color=color, alpha=0.7, edgecolor='black')    ax.set_xlabel(metric, fontsize=11)    ax.set_title(metric, fontweight='bold', fontsize=13)    ax.grid(alpha=0.3, axis='x')    ax.set_xlim(0, 1.05)        # Añadir valores    for bar, val in zip(bars, values):        if not np.isnan(val):            width = bar.get_width()            ax.text(width + 0.02, bar.get_y() + bar.get_height()/2,                   f'{val:.3f}', ha='left', va='center', fontsize=9, fontweight='bold')plt.suptitle('Comparación de Clasificadores - Todas las Métricas',             fontsize=16, fontweight='bold')plt.tight_layout()plt.show()

---# Parte 4: Mejores Prácticas en ValidaciónCómo evitar errores comunes y obtener evaluaciones confiables.

## 4.1 Data Leakage: El Enemigo SilenciosoLa fuga de información invalida la evaluación.

In [ ]:
print("="*80)print("EJEMPLO DE DATA LEAKAGE")print("="*80)# Crear dataset simpleX_leak, y_leak = make_classification(n_samples=200, n_features=10, random_state=42)print(f"\n--- INCORRECTO: Escalar antes de dividir ---")# ❌ MAL: Escalar con todo el datasetscaler_wrong = StandardScaler()X_scaled_wrong = scaler_wrong.fit_transform(X_leak)# Luego dividirX_train_wrong, X_test_wrong, y_train_wrong, y_test_wrong = train_test_split(    X_scaled_wrong, y_leak, test_size=0.3, random_state=42)# Entrenar y evaluarmodel_wrong = LogisticRegression(max_iter=1000)model_wrong.fit(X_train_wrong, y_train_wrong)acc_wrong = model_wrong.score(X_test_wrong, y_test_wrong)print(f"Accuracy (con leakage): {acc_wrong:.4f}")print(f"⚠️  El scaler vio TODO el dataset → información del test filtró al train")print(f"\n--- CORRECTO: Escalar solo con training ---")# ✅ BIEN: Primero dividirX_train_right, X_test_right, y_train_right, y_test_right = train_test_split(    X_leak, y_leak, test_size=0.3, random_state=42)# Luego escalar solo con trainingscaler_right = StandardScaler()X_train_scaled = scaler_right.fit_transform(X_train_right)X_test_scaled = scaler_right.transform(X_test_right)  # Solo transform, NO fit_transform# Entrenar y evaluarmodel_right = LogisticRegression(max_iter=1000)model_right.fit(X_train_scaled, y_train_right)acc_right = model_right.score(X_test_scaled, y_test_right)print(f"Accuracy (sin leakage): {acc_right:.4f}")print(f"✓ El scaler solo vio training → evaluación válida")print(f"\nDiferencia: {acc_wrong - acc_right:.4f}")print(f"La evaluación con leakage es artificialmente optimista!")

## 4.2 Usando Pipelines para Evitar LeakageLos pipelines automatizan el flujo correcto.

In [ ]:
from sklearn.pipeline import Pipeline# Pipeline correctopipeline = Pipeline([    ('scaler', StandardScaler()),    ('classifier', LogisticRegression(max_iter=1000))])# Cross-validation con pipelinecv_scores = cross_val_score(pipeline, X_leak, y_leak, cv=5, scoring='accuracy')print("="*80)print("PIPELINE + CROSS-VALIDATION")print("="*80)print(f"\nAccuracy por fold:")for i, score in enumerate(cv_scores, 1):    print(f"  Fold {i}: {score:.4f}")print(f"\nMedia: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")print(f"\n✓ El pipeline garantiza que el escalamiento se hace DENTRO de cada fold")print(f"✓ No hay leakage entre training y test en ningún fold")

## 4.3 Selección de Métricas Según ContextoNo existe una métrica universalmente superior.

In [ ]:
print("="*80)print("GUÍA DE SELECCIÓN DE MÉTRICAS")print("="*80)print("""📊 REGRESIÓN:  Métrica    | Cuándo usar                        | Sensibilidad a outliers  -----------|------------------------------------|-----------------------  MAE        | Errores en escala original         | Baja (lineal)  MSE        | Penalizar errores grandes          | Alta (cuadrática)  RMSE       | MSE en escala original             | Alta (cuadrática)  R²         | Proporción de varianza explicada   | Media🎯 CLASIFICACIÓN (Binaria):  Escenario                      | Métrica prioritaria  -------------------------------|--------------------  Clases balanceadas             | Accuracy, F1-Score  Clases desbalanceadas          | F1, MCC, ROC-AUC  Minimizar falsos positivos     | Precision  Minimizar falsos negativos     | Recall (Sensitivity)  Balance precision-recall       | F1-Score  Todas las celdas de CM         | MCC  Evaluar umbral variable        | ROC-AUC⚕️ EJEMPLOS PRÁCTICOS:  Problema                | Por qué                           | Métrica  ------------------------|-----------------------------------|----------  Diagnóstico cáncer      | FN son críticos (no detectar)     | Recall  Detección fraude        | FP son costosos (falsa alarma)    | Precision  Spam filtering          | Balance FP y FN                   | F1-Score  Clases muy desbalanceadas| Accuracy engañosa                | MCC, F1  ⚠️  IMPORTANTE:  • Accuracy es engañosa con desbalance (puede ser alta prediciendo siempre la mayoría)  • Precision y Recall son opuestos (mejora de uno empeora el otro)  • F1 balancea Precision y Recall  • MCC es robusto con cualquier distribución de clases  • ROC-AUC evalúa todos los posibles umbrales""")

---# Resumen y Conclusiones## ✅ Lo que hemos aprendido### 1. Partición de Datos* **Hold-out simple**: Rápido pero con alta varianza* **k-Fold CV**: Estándar práctico (k=5 típicamente)* **Leave-One-Out**: Bajo sesgo, alta varianza, costoso* **Estratificación**: Esencial con clases desbalanceadas### 2. Métricas de Regresión* **MAE**: Robusto a outliers, escala original* **MSE**: Penaliza errores grandes, sensible a outliers* **RMSE**: MSE en escala original* **R²**: Proporción de varianza explicada### 3. Métricas de Clasificación* **Accuracy**: Simple pero engañosa con desbalance* **Precision**: De los positivos predichos, cuántos son correctos* **Recall**: De los positivos reales, cuántos detectamos* **F1-Score**: Balance entre Precision y Recall* **ROC-AUC**: Desempeño agregado en todos los umbrales* **MCC**: Robusto con cualquier distribución de clases### 4. Mejores Prácticas* **Evitar data leakage**: Preprocesar DENTRO de cada fold* **Usar pipelines**: Automatizan el flujo correcto* **Reportar varianza**: El desempeño no es un número fijo* **Seleccionar métrica apropiada**: Según el contexto del problema## 🎯 Principios Clave1. **Sin partición no hay evaluación válida**2. **El error en training es optimista**3. **Cross-validation reduce varianza**4. **La métrica debe reflejar el objetivo del problema**5. **Data leakage invalida la evaluación**## 📚 Próximos PasosEn módulos posteriores veremos:* Optimización de hiperparámetros* Nested cross-validation* Evaluación de modelos complejos* Interpretabilidad de resultados---**¡Excelente trabajo!** 🎉  Has completado el módulo de validación y evaluación de modelos.